<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/AI_Pytorch_Keras/blob/main/MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template
1. import libraries
2. dataset load
3. data preparation / procces
4. model create
5. train the model
6. predict (with test dataset)
7. evaluate the model

In [19]:
# import libraries
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

In [20]:
# dataset load
class MNISTDataset(Dataset):
  def __init__(self, csv_file, transform = None):
    self.data = pd.read_csv(csv_file).values
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img = self.data[idx, 1:].astype(np.uint8).reshape(28, 28, 1)
    label = self.data[idx, 0].astype(np.int64)
    if self.transform:
      img = self.transform(img)
    return img, label



In [21]:
# data preparation / proccesing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=0.5, std=0.5)])

train_dataset = MNISTDataset('./sample_data/mnist_train_small.csv', transform=transform)
test_dataset = MNISTDataset('./sample_data/mnist_test.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [25]:
train_dataset.data.shape


(19999, 785)

In [26]:
for img, label in train_loader:
  print(img.shape)
  break

torch.Size([64, 1, 28, 28])


In [29]:
#model create
class CNNPytorch(torch.nn.Module):
  def __init__(self, num_classes = 10):
    super(CNNPytorch, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
    self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3)
    self.pool = torch.nn.MaxPool2d(kernel_size=3)
    self.flatten = torch.nn.Flatten()
    self.fc = torch.nn.Linear(64*5*5, num_classes)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = self.flatten(x)
    x = self.fc(x)
    return torch.softmax(x, dim=1)


model = CNNPytorch()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [28]:
model

CNNPytorch(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=1600, out_features=10, bias=True)
)